# Data preprocessing

This file contains the code for data preprocessing and the description of the obtained dataset.

## Dataset description

The dataset is saved as `/data/dataset.parquet` file with all necessary data for each student, who have either submitted CV, Motivational letter or Presentation. The dataset contains 577 rows and 25 columns in total. The table below gives explicit description of the dataset content.


### Data overview

| # | Column                      | Non-Null Count | Dtype   | Range      | Description                                                                 |
|---|-----------------------------|----------------|---------|------------|-----------------------------------------------------------------------------|
| 0 | cv                          | 567 non-null   | object  | N/A        | The anonymized CV in the form of string.                                    |
| 1 | letter                      | 568 non-null   | object  | N/A        | The anonymized Motivational letter in the form of string.                   |
| 2 | presentation                | 564 non-null   | object  | N/A        | The anonymized text from Presentation slides in the form of string.        |
| 3 | cv_phd_1                    | 567 non-null   | float64 | 0 - 5      | The CV score from first PhD student assessor.                               |
| 4 | cv_phd_2                    | 567 non-null   | float64 | 0 - 5      | The CV score from second PhD student assessor.                              |
| 5 | letter_phd_1                | 567 non-null   | float64 | 0 - 5      | The Motivational letter score from first PhD student assessor.              |
| 6 | letter_phd_2                | 567 non-null   | float64 | 0 - 5      | The Motivational letter score from second PhD student assessor.             |
| 7 | pres_phd_1                  | 567 non-null   | float64 | 0 - 20     | The Presentation score from first PhD student assessor.                     |
| 8 | pres_phd_2                  | 567 non-null   | float64 | 0 - 20     | The Presentation score from second PhD student assessor.                    |
| 9 | pres_class                  | 567 non-null   | float64 | 0 - 3      | The type of Presentation (review, article replication of personal project). |
| 10| video_phd_1                 | 383 non-null   | float64 | 0 - 20     | The Presentation video score from first PhD student assessor.               |
| 11| video_phd_2                 | 409 non-null   | float64 | 0 - 20     | The Presentation video score from second PhD student assessor.              |
| 12| all_phd_1                   | 567 non-null   | float64 | 0 - 100    | The total score from first PhD student assessor.                            |
| 13| all_phd_2                   | 567 non-null   | float64 | 0 - 100    | The total score from first PhD student assessor.                            |
| 14| final_human_score           | 98 non-null    | float64 | 0 - 100    | The final score (weighted mean of PhD students and Professor scores).       |
| 15| prof_score                  | 98 non-null    | float64 | 0 - 10     | The Professor's overall score.                                              |
| 16| offline_test_1              | 48 non-null    | float64 | 0 - 1      | The results of the first offline test.                                      |
| 17| offline_test_2              | 48 non-null    | float64 | 0 - 1      | The results of the second offline test.                                     |
| 18| offline_test_3              | 48 non-null    | float64 | 0 - 1      | The results of the third offline test.                                      |
| 19| offline_test_4              | 48 non-null    | float64 | 0 - 1      | The results of the fourth offline test.                                     |
| 20| offline_test_total          | 48 non-null    | float64 | 0 - 1      | The sum of all offline tests.                                               |
| 21| online_test_score           | 68 non-null    | float64 | 0 - 67     | The results of the online test                                              |
| 22| project_participation_flag  | 577 non-null   | bool    | T/F        | The flag of student's participation in the project activity.                |
| 23| project_ta_score            | 118 non-null   | float64 | 0 - 5      | The project score from the teacher assistants.                              |
| 24| project_peer_score          | 118 non-null   | float64 | 0 - 10     | The project score from the peer-review stage.                               |

### Data acquisition

The first three conlumns contain input anonymized data: the text of CV, Motivational letter and text extracted from Presentation slides. 

The columns with `phd` interfix contain the scores for every type of the data that was submitted by student during the application period. As each student application has been evaluated by two PhD students independently, the columns also contain numeric suffix to separate the assessors. 

After being evaluated by PhD students, top 100 applicants have been evaluated by Professors; this score is preseneted into `prof_score` column. The final score denoted as `final_human_score` is the weighted sum of two PhD students evaluations and Professors score.

The columns with the `offline_test` prefix contain results of the intermediate tests that have been taken by the offline participants. If the offline participant have missed the test, the test score is zeros. For online participants the scores are NaNs.

The `online_test_score` columns contains the scores of online and offline participants for final test. The score ranges from 0 to 67. All the applicants who missed the test have NaN values by default.

The `project_participation_flag` indicates if the applicant have participated in final projects activity or not.

The `project_ta_score` and `project_peer_score` contains the scores for the project, given by teacher assistants and by other school participants respectively.

## Preprocessing pipeline

In [1]:
import os

import pandas as pd
from tqdm import tqdm

In [2]:
JOIN_TYPE = "left"

### Unicode names restoration

In [3]:
import re


def unescape_filename(name):
    """
    Replace all occurrences of '#U' followed by four hexadecimal characters
    with the corresponding Unicode character.
    """
    return re.sub(r"#U([0-9a-fA-F]{4})", lambda m: chr(int(m.group(1), 16)), name)


In [4]:
FOLDER = "./data/raw"

for dirpath, dirnames, filenames in os.walk(FOLDER):
    if dirnames == []:
        for filename in filenames:
            new_filename = os.path.join(dirpath, unescape_filename(filename).strip())
            old_filename = os.path.join(dirpath, filename)
            os.rename(old_filename, new_filename)


### Student names unification

In [5]:
students = set()

for _, _, filenames in os.walk(FOLDER):
    filenames = set(name[: name.rfind(".txt")] for name in filenames)
    students = students.union(filenames)

dataset = pd.DataFrame(index=sorted(students))
dataset.index.name = "name"
print(f"Unique students: {len(dataset)}")

Unique students: 574


### Data writing

In [6]:
for dirpath, dirnames, filenames in os.walk(FOLDER):
    if dirnames:
        continue

    subfolder = os.path.basename(dirpath)
    label = subfolder[: subfolder.find("s_")]
    for filename in tqdm(filenames, f"Processing files from {dirpath}"):
        with open(os.path.join(dirpath, filename), encoding="utf-8") as f:
            content = f.read()
        idx = filename[: filename.rfind(".txt")]
        dataset.loc[idx, label] = content

dataset.info()

Processing files from ./data/raw\letters_an: 100%|██████████| 565/565 [00:07<00:00, 75.72it/s]
Processing files from ./data/raw\presentations_an: 100%|██████████| 561/561 [00:07<00:00, 71.95it/s]

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cv            564 non-null    object
 1   letter        565 non-null    object
 2   presentation  561 non-null    object
dtypes: object(3)
memory usage: 34.1+ KB


In [7]:
def excel_to_python_float(x):
    try:
        return float(str(x).replace(",", "."))
    except ValueError:
        return None


def excel_to_python_bool(x):
    if x == "ИСТИНА":
        return 1
    elif x == "ЛОЖЬ":
        return 0
    else:
        return None


def clean_indexes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the index of the DataFrame by removing leading and trailing whitespace
    and converting to string.
    """
    df.index = df.index.map(lambda x: str(x).strip())
    return df[~df.index.isna()]

### PhD students' evluations (round 1)

In [8]:
phd_evals = pd.read_csv(os.path.join(FOLDER, "student_evaluation_main.csv"), sep=";")
phd_evals = phd_evals.set_index("new_names")
phd_evals = clean_indexes(phd_evals)

# Rename columns for consistency
mapper = {
    col: col.replace("mean", "phd").replace("points", "phd")
    for col in phd_evals.columns
}
phd_evals = phd_evals.rename(columns=mapper)
phd_evals = phd_evals[~phd_evals["pres_class_2"].isna()]
if all(phd_evals["pres_class_1"] == phd_evals["pres_class_2"]):
    print("Presentation classes are equal. Perform reduction")
    phd_evals = phd_evals.drop(columns="pres_class_2")
    phd_evals = phd_evals.rename(columns={"pres_class_1": "pres_class"})


# Convert Excel-style floats to Pythonic
phd_evals = phd_evals.map(excel_to_python_float)

phd_evals.info()

Presentation classes are equal. Perform reduction
<class 'pandas.core.frame.DataFrame'>
Index: 609 entries, Martin Slobodník to Edmund Štefanko
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cv_phd_1      609 non-null    float64
 1   cv_phd_2      609 non-null    float64
 2   letter_phd_1  609 non-null    float64
 3   letter_phd_2  609 non-null    float64
 4   pres_phd_1    609 non-null    float64
 5   pres_phd_2    609 non-null    float64
 6   pres_class    609 non-null    float64
 7   video_phd_1   409 non-null    float64
 8   video_phd_2   439 non-null    float64
 9   all_phd_1     609 non-null    float64
 10  all_phd_2     609 non-null    float64
dtypes: float64(11)
memory usage: 57.1+ KB


In [9]:
dataset = pd.merge(dataset, phd_evals, left_index=True, right_index=True, how=JOIN_TYPE)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cv            564 non-null    object 
 1   letter        565 non-null    object 
 2   presentation  561 non-null    object 
 3   cv_phd_1      564 non-null    float64
 4   cv_phd_2      564 non-null    float64
 5   letter_phd_1  564 non-null    float64
 6   letter_phd_2  564 non-null    float64
 7   pres_phd_1    564 non-null    float64
 8   pres_phd_2    564 non-null    float64
 9   pres_class    564 non-null    float64
 10  video_phd_1   380 non-null    float64
 11  video_phd_2   406 non-null    float64
 12  all_phd_1     564 non-null    float64
 13  all_phd_2     564 non-null    float64
dtypes: float64(11), object(3)
memory usage: 83.4+ KB


### Professors' evaluations (round 2)

In [10]:
prof_evals = pd.read_csv(os.path.join(FOLDER, "prof_eval_anon.csv"), sep=";")
prof_evals = prof_evals.set_index("names")
prof_evals = clean_indexes(prof_evals)

# Rename columns for consistency
prof_evals = prof_evals.drop(columns="initial marks (mean of two)")
prof_evals = prof_evals.dropna(axis="columns")

mapper = {
    "final mark (weighted)": "final_human_score",
    "prof marks (normilized)": "prof_score",
}
prof_evals = prof_evals.rename(columns=mapper)

# Convert Excel-style floats to Pythonic
prof_evals = prof_evals.map(excel_to_python_float)

prof_evals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, Emilia Pop to Ante Stoll
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   final_human_score  102 non-null    float64
 1   prof_score         102 non-null    float64
dtypes: float64(2)
memory usage: 2.4+ KB


In [11]:
dataset = pd.merge(
    dataset, prof_evals, left_index=True, right_index=True, how=JOIN_TYPE
)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   cv                 564 non-null    object 
 1   letter             565 non-null    object 
 2   presentation       561 non-null    object 
 3   cv_phd_1           564 non-null    float64
 4   cv_phd_2           564 non-null    float64
 5   letter_phd_1       564 non-null    float64
 6   letter_phd_2       564 non-null    float64
 7   pres_phd_1         564 non-null    float64
 8   pres_phd_2         564 non-null    float64
 9   pres_class         564 non-null    float64
 10  video_phd_1        380 non-null    float64
 11  video_phd_2        406 non-null    float64
 12  all_phd_1          564 non-null    float64
 13  all_phd_2          564 non-null    float64
 14  final_human_score  95 non-null     float64
 15  prof_score         95 non-null     float64
dtypes: float

### Offline tests

In [12]:
offline_test = pd.read_csv(os.path.join(FOLDER, "offline_tests.csv"))
offline_test = offline_test.set_index("names")
offline_test = clean_indexes(offline_test)

# Rename columns for consistency
mapper = {"Total": "offline_test_total"}
idx = 1
for col in offline_test.columns:
    if col.find("Jul") > 0:
        mapper[col] = f"offline_test_{idx}"
        idx += 1
offline_test = offline_test.rename(columns=mapper)

# Set zeros for student who had been present but did not take the offline test
offline_test = offline_test.fillna(0)

offline_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, Romana Rysová to Valentina Toma
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   offline_test_1      54 non-null     float64
 1   offline_test_2      54 non-null     float64
 2   offline_test_3      54 non-null     float64
 3   offline_test_4      54 non-null     float64
 4   offline_test_total  54 non-null     float64
dtypes: float64(5)
memory usage: 2.5+ KB


In [13]:
dataset = pd.merge(
    dataset, offline_test, left_index=True, right_index=True, how=JOIN_TYPE
)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cv                  567 non-null    object 
 1   letter              568 non-null    object 
 2   presentation        564 non-null    object 
 3   cv_phd_1            567 non-null    float64
 4   cv_phd_2            567 non-null    float64
 5   letter_phd_1        567 non-null    float64
 6   letter_phd_2        567 non-null    float64
 7   pres_phd_1          567 non-null    float64
 8   pres_phd_2          567 non-null    float64
 9   pres_class          567 non-null    float64
 10  video_phd_1         383 non-null    float64
 11  video_phd_2         409 non-null    float64
 12  all_phd_1           567 non-null    float64
 13  all_phd_2           567 non-null    float64
 14  final_human_score   98 non-null     float64
 15  prof_score          98 non-null     flo

### Online test and projects participation

In [14]:
online_test = pd.read_csv(
    os.path.join(FOLDER, "anonymized_test_project_scores_final.csv"), sep=";"
)
online_test = online_test.set_index("new_names")
online_test = clean_indexes(online_test)

mapper = {
    "Score": "online_test_score",
    "project_flag": "project_participation_flag",
}
online_test = online_test.rename(columns=mapper)
online_test["project_participation_flag"] = online_test[
    "project_participation_flag"
].map(excel_to_python_bool)

online_test = online_test.fillna(0)

online_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71 entries, Günter Kroker B.Sc. to Varvara Ababei
Data columns (total 2 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   online_test_score           71 non-null     int64
 1   project_participation_flag  71 non-null     int64
dtypes: int64(2)
memory usage: 1.7+ KB


In [15]:
dataset = pd.merge(
    dataset, online_test, left_index=True, right_index=True, how=JOIN_TYPE
)
dataset["project_participation_flag"] = dataset["project_participation_flag"].fillna(0)
dataset["project_participation_flag"] = dataset["project_participation_flag"].astype(
    bool
)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cv                          567 non-null    object 
 1   letter                      568 non-null    object 
 2   presentation                564 non-null    object 
 3   cv_phd_1                    567 non-null    float64
 4   cv_phd_2                    567 non-null    float64
 5   letter_phd_1                567 non-null    float64
 6   letter_phd_2                567 non-null    float64
 7   pres_phd_1                  567 non-null    float64
 8   pres_phd_2                  567 non-null    float64
 9   pres_class                  567 non-null    float64
 10  video_phd_1                 383 non-null    float64
 11  video_phd_2                 409 non-null    float64
 12  all_phd_1                   567 non-null    float64
 13  all_phd_2        

### Projects scores

In [16]:
raw_projects = pd.read_csv(os.path.join(FOLDER, "anon_projects_scores.csv"), sep=",")
raw_projects = raw_projects.set_index("Anon Names")
raw_projects = clean_indexes(raw_projects)
raw_projects = raw_projects.drop(columns="Project name ")

raw_projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, Tania Stoica, Žofia Sameľová, Univ.Prof. Elke Sontag, Vratislav Repiský, Hans-Martin Jungfer to nan
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Final Score             39 non-null     int64  
 1   Students Scores (0-10)  38 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 936.0+ bytes


In [17]:
projects = pd.DataFrame(columns=raw_projects.columns, dtype=float)

for team in raw_projects.index:
    participants = map(lambda x: x.strip(), str(team).split(","))
    for participant in participants:
        projects.loc[participant, raw_projects.columns] = raw_projects.loc[team, :]

mapper = {
    "Final Score": "project_ta_score",
    "Students Scores (0-10)": "project_peer_score",
}
projects = projects.rename(columns=mapper)

projects.info()

<class 'pandas.core.frame.DataFrame'>
Index: 128 entries, Tania Stoica to nan
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   project_ta_score    128 non-null    float64
 1   project_peer_score  127 non-null    float64
dtypes: float64(2)
memory usage: 7.1+ KB


In [18]:
dataset = pd.merge(dataset, projects, left_index=True, right_index=True, how=JOIN_TYPE)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, Adam Mazilescu to Ștefania Albu
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cv                          567 non-null    object 
 1   letter                      568 non-null    object 
 2   presentation                564 non-null    object 
 3   cv_phd_1                    567 non-null    float64
 4   cv_phd_2                    567 non-null    float64
 5   letter_phd_1                567 non-null    float64
 6   letter_phd_2                567 non-null    float64
 7   pres_phd_1                  567 non-null    float64
 8   pres_phd_2                  567 non-null    float64
 9   pres_class                  567 non-null    float64
 10  video_phd_1                 383 non-null    float64
 11  video_phd_2                 409 non-null    float64
 12  all_phd_1                   567 non-null    float64
 13  all_phd_2        

In [19]:
FILE_NAME = "./data/dataset.parquet"

dataset.to_parquet(FILE_NAME)

### Final check

It has been found that the `.parquet` format is the optimal format for data saving because it is not distorted by commas and semicolumns which are presented in the textual data.

In [20]:
check = pd.read_parquet(FILE_NAME)
assert check.equals(dataset), "Dataset saved incorrectly. Check the data consistency!"
del check